In [3]:
import pandas as pd
import numpy as np
import joblib 

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

import lightgbm as lgb
import re


In [4]:
#binary classification
bc_adults = pd.read_csv('binary classification/datasets/adults/adults_train.csv')
X_bc_adults = bc_adults.iloc[:,:-1]
y_bc_adults = bc_adults.iloc[:,-1]

bc_german = pd.read_csv('binary classification/datasets/german credit/german_credit_train.csv')
X_bc_german = bc_german.iloc[:,:-1]
y_bc_german = bc_german.iloc[:,-1]

In [5]:
#regression
reg = pd.read_csv('regression/datasets/insurance/insurance_train.csv')
X_reg = reg.iloc[:,:-1]
y_reg = reg.iloc[:,-1]

In [6]:
#multiclass classification
mc = pd.read_csv('multiclass classification/datasets/healthcare/healthcare_train.csv')
X_mc = mc.iloc[:,:-1]
y_mc = mc.iloc[:,-1]

In [7]:
#clustering
clust = pd.read_csv('clustering/datasets/customers/customers.csv')
X_clust = clust

In [6]:
pip list

Package                                  Version
---------------------------------------- ------------------
absl-py                                  2.1.0
aiohttp                                  3.9.3
aiosignal                                1.3.1
annotated-types                          0.6.0
anyio                                    4.2.0
appdirs                                  1.4.4
appnope                                  0.1.3
asgiref                                  3.7.2
asttokens                                2.4.1
astunparse                               1.6.3
attrs                                    23.2.0
azure-core                               1.30.0
azure-identity                           1.15.0
backoff                                  2.2.1
bcrypt                                   4.1.2
beautifulsoup4                           4.12.3
bert-score                               0.3.13
bs4                                      0.0.2
build                                   

In [7]:
# Initialize and train LightGBM for binary classification, sklearn wrapper and booster
model = lgb.LGBMClassifier(objective='binary',
                         boosting_type='gbdt',
                         num_leaves=31,
                         learning_rate=0.05,
                         feature_fraction=0.9)
model.fit(X_bc_adults, y_bc_adults)
joblib.dump(model, 'binary classification/models/lightgbm-4_2_0_sklearn-binary_classification_adults.joblib')

train_data = lgb.Dataset(X_bc_adults, label=y_bc_adults)
lgb_params = {
    'objective': 'binary',
    'metric': 'binary_error',  
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'max_depth': -1
}
bst = lgb.train(lgb_params, train_data, 100)
print(bst.predict(X_bc_adults))
joblib.dump(bst, 'binary classification/models/lightgbm-4_2_0-binary_classification_adults.joblib')



[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Number of positive: 9244, number of negative: 29756
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000364 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 367
[LightGBM] [Info] Number of data points in the train set: 39000, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.237026 -> initscore=-1.169056
[LightGBM] [Info] Start training from score -1.169056
[0.02746073 0.28400039 0.14518856 ... 0.04014923 0.00438456 0.08562547]


['binary classification/models/lightgbm-4_2_0-binary_classification_adults.joblib']

In [8]:
# Initialize and train LightGBM for binary classification, sklearn wrapper and booster
model = lgb.LGBMClassifier(objective='binary',
                         boosting_type='gbdt',
                         num_leaves=31,
                         learning_rate=0.05,
                         feature_fraction=0.9)
model.fit(X_bc_german, y_bc_german)
joblib.dump(model, 'binary classification/models/lightgbm-4_2_0_sklearn-binary_classification_german.joblib')

train_data = lgb.Dataset(X_bc_german, label=y_bc_german)
lgb_params = {
    'objective': 'binary',
    'metric': 'binary_error',  
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'max_depth': -1
}
bst = lgb.train(lgb_params, train_data, 100)
print(bst.predict(X_bc_german))
joblib.dump(bst, 'binary classification/models/lightgbm-4_2_0-binary_classification_german.joblib')

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Number of positive: 247, number of negative: 553
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000925 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 459
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.308750 -> initscore=-0.805970
[LightGBM] [Info] Start training from score -0.805970
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

['binary classification/models/lightgbm-4_2_0-binary_classification_german.joblib']

In [9]:
# Initialize and train LightGBM for regression, sklearn wrapper and booster
model = lgb.LGBMRegressor(objective='regression',
                        boosting_type='gbdt',
                        num_leaves=31,
                        learning_rate=0.05,
                        feature_fraction=0.9)
model.fit(X_reg, y_reg)
joblib.dump(model, 'regression/models/lightgbm-4_2_0_sklearn-regression_insurance.joblib')

train_data = lgb.Dataset(X_reg, label=y_reg)
lgb_params = {
    'objective': 'regression',  
    'metric': 'rmse',            
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
bst = lgb.train(lgb_params, train_data, 100)
joblib.dump(bst, 'regression/models/lightgbm-4_2_0-regression_insurance.joblib')


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000345 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 315
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 7
[LightGBM] [Info] Start training from score 13173.332294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

['regression/models/lightgbm-4_2_0-regression_insurance.joblib']

In [10]:
# Initialize and train LightGBM for multiclass classification, sklearn wrapper and booster
model = lgb.LGBMClassifier(objective='multiclass',
                         num_class=3,  # Number of classes in your dataset
                         boosting_type='gbdt',
                         num_leaves=31,
                         learning_rate=0.05,
                         feature_fraction=0.9)
model.fit(X_mc, y_mc)
joblib.dump(model, 'multiclass classification/models/lightgbm-4_2_0_sklearn-multiclass_classification_healthcare.joblib')

train_data = lgb.Dataset(X_mc, label=y_mc)
lgb_params = {
    'objective': 'multiclass',  
    'num_class': len(set(y_mc)),              
    'metric': 'multi_logloss',   
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
bst = lgb.train(lgb_params, train_data, 100)
joblib.dump(bst, 'multiclass classification/models/lightgbm-4_2_0-multiclass_classification_healthcare.joblib')


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000981 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 596
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 10
[LightGBM] [Info] Start training from score -1.068932
[LightGBM] [Info] Start training from score -1.114742
[LightGBM] [Info] Start training from score -1.112838
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


['multiclass classification/models/lightgbm-4_2_0-multiclass_classification_healthcare.joblib']

In [11]:
# Initialize and train sklearn for binary classification
model = LogisticRegression(solver='liblinear')
model.fit(X_bc_adults, y_bc_adults)
joblib.dump(model, 'binary classification/models/scikit-learn-1_3_2-binary_classification_adults.joblib')

['binary classification/models/scikit-learn-1_3_2-binary_classification_adults.joblib']

In [12]:
# Initialize and train sklearn for binary classification
model = LogisticRegression(solver='liblinear')
model.fit(X_bc_german, y_bc_german)
joblib.dump(model, 'binary classification/models/scikit-learn-1_3_2-binary_classification_german.joblib')

['binary classification/models/scikit-learn-1_3_2-binary_classification_german.joblib']

In [13]:
# Initialize and train sklearn for regression
model = LinearRegression()
model.fit(X_reg, y_reg)
joblib.dump(model, 'regression/models/scikit-learn-1_3_2-regression_insurance.joblib')

['regression/models/scikit-learn-1_3_2-regression_insurance.joblib']

In [14]:
# Initialize and train sklearn for multiclass classification
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_mc, y_mc)
joblib.dump(model, 'multiclass classification/models/scikit-learn-1_3_2-multiclass_classification_healthcare.joblib')

['multiclass classification/models/scikit-learn-1_3_2-multiclass_classification_healthcare.joblib']

In [15]:
# Initialize and train sklearn for clustering
model = KMeans(n_clusters=5, random_state=42)
model.fit(X_clust)
joblib.dump(model, 'clustering/models/scikit-learn-1_3_2-clustering_customers.joblib')

['clustering/models/scikit-learn-1_3_2-clustering_customers.joblib']

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
# Initialize and train PyTorch model for binary classification
X_train = torch.Tensor(X_bc_adults.values)
y_train = torch.Tensor(y_bc_adults)
# Define a simple logistic regression model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1),
    nn.Sigmoid()
)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs = 20
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model,'binary classification/models/torch-2_2_1-binary_classification_adults.pth')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model_name = "binary classification/models/torch-2_2_1-binary_classification_adults.pth"
model = torch.load(model_name)
df_test = pd.read_csv("binary classification/datasets/adults/adults_test.csv")
tensor = torch.Tensor(df_test.drop(columns=["class"]).values)
output = model(tensor)
predicted_classes = (output > 0.5).int()  # Use threshold 0.5 for binary classification


In [19]:
# Initialize and train PyTorch model for binary classification
X_train = torch.Tensor(X_bc_german.values)
y_train = torch.Tensor(y_bc_german)
# Define a simple logistic regression model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1),
    nn.Sigmoid()
)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model,'binary classification/models/torch-2_2_1-binary_classification_german.pth')

Epoch [100/1000], Loss: 30.8750
Epoch [200/1000], Loss: 30.8750
Epoch [300/1000], Loss: 30.8750
Epoch [400/1000], Loss: 30.8750
Epoch [500/1000], Loss: 30.8750
Epoch [600/1000], Loss: 30.8750
Epoch [700/1000], Loss: 30.8750
Epoch [800/1000], Loss: 30.8750
Epoch [900/1000], Loss: 30.8750
Epoch [1000/1000], Loss: 30.8750


In [20]:
model_name = "binary classification/models/torch-2_2_1-binary_classification_german.pth"
model = torch.load(model_name)
df_test = pd.read_csv("binary classification/datasets/german credit/german_credit_test.csv")
tensor = torch.Tensor(df_test.drop(columns=["default"]).values)
output = model(tensor)
predicted_classes = (output > 0.5).int()  # Use threshold 0.5 for binary classification


In [26]:
# Initialize and train PyTorch model for regression
X_train = torch.FloatTensor(X_reg.values)
y_train = torch.FloatTensor(y_reg)
# Define a simple regression model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1)
)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#torch.save(model.state_dict(), 'models/torch-2_3_0-regression.joblib')
torch.save(model,'regression/models/torch-2_2_1-regression_insurance.pth')

Epoch [100/10000], Loss: 315870528.0000
Epoch [200/10000], Loss: 313738144.0000
Epoch [300/10000], Loss: 311623520.0000
Epoch [400/10000], Loss: 309526304.0000
Epoch [500/10000], Loss: 307446272.0000
Epoch [600/10000], Loss: 305383168.0000
Epoch [700/10000], Loss: 303336640.0000
Epoch [800/10000], Loss: 301306592.0000
Epoch [900/10000], Loss: 299292704.0000
Epoch [1000/10000], Loss: 297294720.0000
Epoch [1100/10000], Loss: 295312512.0000
Epoch [1200/10000], Loss: 293345760.0000
Epoch [1300/10000], Loss: 291394304.0000
Epoch [1400/10000], Loss: 289457952.0000
Epoch [1500/10000], Loss: 287536448.0000
Epoch [1600/10000], Loss: 285629632.0000
Epoch [1700/10000], Loss: 283737408.0000
Epoch [1800/10000], Loss: 281859488.0000
Epoch [1900/10000], Loss: 279995776.0000
Epoch [2000/10000], Loss: 278146048.0000
Epoch [2100/10000], Loss: 276310176.0000
Epoch [2200/10000], Loss: 274488064.0000
Epoch [2300/10000], Loss: 272679488.0000
Epoch [2400/10000], Loss: 270884384.0000
Epoch [2500/10000], Loss:

In [25]:
# Initialize and train PyTorch model for multiclass classification
X_train = torch.FloatTensor(X_mc.values)
y_train = torch.LongTensor(y_mc)
# Define a simple multiclass classification model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 3)  # 3 output neurons for three classes
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#torch.save(model.state_dict(), 'models/torch-2_3_0-multiclass_classification.joblib')
torch.save(model,'multiclass classification/models/torch-2_2_1-multiclass_classification_healthcare.pth')

Epoch [100/1000], Loss: 2665888.5000
Epoch [200/1000], Loss: 961548.4375
Epoch [300/1000], Loss: 2530439.5000
Epoch [400/1000], Loss: 4044299.5000
Epoch [500/1000], Loss: 2313611.0000
Epoch [600/1000], Loss: 852011.0625
Epoch [700/1000], Loss: 2421012.2500
Epoch [800/1000], Loss: 3546060.2500
Epoch [900/1000], Loss: 1815501.8750
Epoch [1000/1000], Loss: 663847.3125
